# Model for Question 6

In [8]:
import json
import random
import re
import os
import sys
import collections
import nltk
import tqdm
import itertools
import string
import math
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import scipy as sp
from scipy import sparse
import os.path
import unicodedata

from tqdm import tqdm, trange
from collections import Counter
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer

tokenizer = RegexpTokenizer(r'\w+')

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, LSTM, Multiply

from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.recurrent import LSTM
from keras.layers.merge import concatenate
from keras.layers import Add, Subtract
import keras_metrics

from keras.callbacks import ModelCheckpoint, EarlyStopping

np.random.seed(7)

In [3]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

modelw = loadGloveModel('glove.6B.50D.txt')

Loading Glove Model
Done. 400000  words loaded!


### Code cell to remove full stops

In [5]:
claims6 = pickle.load(open( "pkl_files/claims6.p", "rb" ))
evid6 = pickle.load(open( "pkl_files/evid6.p", "rb" ))
label6 = pickle.load(open( "pkl_files/label6.p", "rb" ))

claimstest = pickle.load(open( "pkl_files/claimstest.p", "rb" ))
evidtest = pickle.load(open( "pkl_files/evidtest.p", "rb" ))
labeltest = pickle.load(open( "pkl_files/labeltest.p", "rb" ))

claims_temp = []

for claim in claims6:
    last_word = claim[-1]
    temp_claim = claim[:-1]
    temp_claim.append(last_word[:-1])
    claims_temp.append(temp_claim)
    
claims6 = claims_temp

claims_temp = []

for claim in claimstest:
    last_word = claim[-1]
    temp_claim = claim[:-1]
    temp_claim.append(last_word[:-1])
    claims_temp.append(temp_claim)
    
claimstest = claims_temp

### Creating Vector representation

In [11]:
def rep_maker(claims, evids):
    
    claim_arrays = []
    evid_arrays = []

    for claim in claims:
        claim_array = np.zeros(50)
        for word in claim:
            try:
                claim_array += modelw[word] 
            except:
                pass
        claim_arrays.append(claim_array)

    for evid in evids:
        evid_array = np.zeros(50)
        for word in evid:
            try:
                evid_array += modelw[word]     
            except:
                pass
        evid_arrays.append(evid_array)
        
    claim_array = np.array(claim_arrays)
    evid_array = np.array(evid_arrays)
    bias = np.ones((len(claim_arrays), 1))
    
    input_matrix = np.concatenate([bias,claim_array, evid_array], axis = 1)
    
    return(input_matrix)

In [12]:
claim_train = rep_maker(claims6, evid6)
claim_test = rep_maker(claimstest, evidtest)
y_train = label6
y_test = labeltest

### Training model

In [13]:
X = claim_train
Y = y_train
# create model
model = Sequential()
model.add(Dense(101, input_dim=101, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', keras_metrics.precision(), keras_metrics.recall()])
# Fit the model
model.fit(X, Y, epochs=10, batch_size=10, validation_split=0.2)
# evaluate the model
scores_train = model.evaluate(X, Y)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 158011 samples, validate on 39503 samples
Epoch 1/10
158011/158011 [==============================] - 33s 207us/step - loss: 0.5497 - acc: 0.7406 - precision: 0.7513 - recall: 0.9559 - val_loss: 0.5207 - val_acc: 0.7523 - val_precision: 0.7676 - val_recall: 0.9442
Epoch 2/10
158011/158011 [==============================] - 33s 207us/step - loss: 0.5034 - acc: 0.7645 - precision: 0.7716 - recall: 0.9557 - val_loss: 0.5062 - val_acc: 0.7661 - val_precision: 0.7863 - val_recall: 0.9300
Epoch 3/10
158011/158011 [==============================] - 33s 212us/step - loss: 0.4833 - acc: 0.7764 - precision: 0.7838 - recall: 0.9516 - val_loss: 0.4991 - val_acc: 0.7692 - val_precision: 0.7986 - val_recall: 0.9114
Epoch 4/10
158011/158011 [==============================] - 33s 209us/step - loss: 0.4702 - acc: 0.7841 - precision: 0.7911 - recall: 0.9510 - val_loss: 0.4923 

In [14]:
scores_train = model.evaluate(X, Y)
scores_test = model.evaluate(claim_test, y_test)

22556/22556 [==============================] - 2s 70us/step


In [15]:
scores_train

[0.43222101540628366, 0.8079781686361474, 0.810673254359896, 0.956980331254993]

In [16]:
scores_test

[0.7722183164499143,
 0.6130076254496529,
 0.5678718337271711,
 0.9337011657832721]